<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <h3> Objective</h3> <br>
    <b style = "color:#000000"><i>To delete evicted pods from namespaces using unSkript actions</i></b>
</div>
</center>
<br>

<center><h2>Delete Evicted Pods From Namespaces</h2></center>

# Steps Overview
1)[ Show All Evicted Pods From Namespace.](#1)<br>
2)[ Delete Evicted Pods From Namespace.](#2)

### Command Selection
In this action, we select the command based on the user input parameter. If the user provides the namespace input, then it only collects pods for the given namespace; otherwise, it will select all pods from all the namespaces.

>Output variable: `command`

In [37]:
command = "kubectl get pods --all-namespaces -o json | grep Evicted"
if namespace:
    command = "kubectl get pods -n " + namespace + " -o json | grep Evicted"

### <a id="1">Show All Evicted Pods From All Namespaces</a>
In this action, we will show the evicted pods from all namespaces by using command "kubectl get pods --all-namespaces | grep Evicted"
>Input parameters: `kubectl_command`

>Output variable: `Pods_Details`

In [44]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "command"
    }''')
task.configure(outputName="pods_details")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

### Command Selection
In this action, we select the command based on the user input parameter. If the user provides the namespace input, then it only collects pods for the given namespace; otherwise, it will select all pods from all the namespaces.

>Output variable: `delete_command`

In [43]:
import json

pods_data = []
if pods_details:
    pod_details = json.loads(pods_details)
    for k, v in pod_details.items():
        if "items" in k:
            for i in v:
                pod_dict = {}
                pod_dict["pod_name"] = i["metadata"]["name"]
                pod_dict["namespace"] = i["metadata"]["namespace"]
                pods_data.append(pod_dict)

delete_command = []
for pod in pods_data:
    command = "kubectl delete pod " + pod["pod_name"] + " -n " + pod["namespace"]
    delete_command.append(command)


### <a id="2">Delete Evicted Pods From All Namespaces</a>
In this action, we will delete the evicted pods from all namespaces by using command "kubectl get pods --all-namespaces | grep Evicted | awk '{print $2 " --namespace=" $1}' | xargs kubectl delete pod"
>Input parameters: `kubectl_command`

>Output variable: `Delete_Status`

In [32]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "kubectl_command": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "delete_command",
    "iter_parameter": "kubectl_command"
    }''')
task.configure(outputName="delete_status")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's Kubernetes actions, and this runbook shows all the evicted pods from all namespaces and deletes them. To view the full platform capabilities of unSkript please visit https://us.app.unskript.io